In [223]:
import numpy as np
from itertools import product, combinations, combinations_with_replacement
from collections import defaultdict
from tqdm import tqdm

#### (Inductive logic programming in the context of prop ositional logic) Le p,q,r be propositional variables (6p)

Since i find it intreating to apply theory to parties i will try to implement the algorithm in python. My argument for that is that if i can implement it then i probably understand it. I will reason for the answers that the algorithm gives.

#### Assignment 1

1. Suppose that all the formulas ¬p ∨ r, ¬q ∨ ¬r and p are true. Determine the
truth values of q and r.

In [3]:
true_statements = [
    lambda p, q, r: not p or r,
    lambda p, q, r: not q or not r,
    lambda p, q, r: p,
]

for p, q, r in product([True, False], repeat=3):
    if all(map(lambda f: f(p, q, r), true_statements)):
        print(f"p={p}, q={q}, r={r}")


p=True, q=False, r=True


* if p is true then r must be true given true statement ¬p ∨ r.
* if r is true then q must be false given true statement ¬q ∨ ¬r.

Answer: q is false and r is true.

2. Construct a CNF φ which avoids N and covers E where:

    N = {(0,1,0),(1,0,0),(1,1,0)} and

    E = {(0,0,0),(0,0,1),(0,1,1),(1,0,1),(1,1,1)}.
    
    Try to simplify φ as much as p ossible by removing unnecessary 1 literals (if such
    exist).

In [4]:
# E = np.array([(1,1,0,1),(1,0,1,0),(0,1,1,0),(0,1,0,1),(1,0,1,1)])
# N = np.array([(0,0,0,0),(1,1,1,1),(1,1,1,0),(0,1,0,0),(1,0,0,1)])
E = np.array([(0,0,0),(0,0,1),(0,1,1),(1,0,1),(1,1,1)])
N = np.array([(0,1,0),(1,0,0),(1,1,0)])
def covers(cause, statements):
    """
    all positive examples satisfy φ
    """
    cause = np.array(cause, dtype=bool)
    covered = np.zeros(len(statements), dtype=bool)
    for i, statement in enumerate(statements):
        res = list(map(lambda c, s: c if s  else not c, cause, statement))
        covered[i] = any(res)
    return all(covered)

def avoids(cause, statements):
    """
    no negative example satisfy φ
    """
    cause = np.array(cause, dtype=bool)
    avoided = np.zeros(len(statements), dtype=bool)
    for i, statement in enumerate(statements):
        res = list(map(lambda c, s: c if s  else not c, cause, statement))
        avoided[i] = all(res)
    return not any(avoided)

def cause_str(cause):
    cause, idxs = cause
    return '(' + ' ∨ '.join(map(lambda i: f'{L[i]}' if cause[i] else f'¬{L[i]}', idxs)) + ')'

L = ['p', 'q', 'r', 's', 't']

def find_H(cause, E, N, maximal=True):
    for cause_size in  sorted(range(1, len(E[0])+1), reverse=maximal):
        for idxs in combinations(range(len(E[0])), cause_size):
            curr_cause = [cause[i] for i in idxs]
            if covers(curr_cause, statements=E[:, idxs]) and avoids(curr_cause, statements=N[:, idxs]):
                return cause, idxs
    return None, None

def algorithm(E, N, maximal=True):
    """
    Algorithm 1; producing a maximally general set or simplified set of clauses H which covers E and avoids N
    (3.3) in Logic_and_probability_7.pdf
    """
    inxs = []
    H = []
    used = set()
    for cause in N:
        cause = [int(not c) for c in cause]
        cause, idxs = find_H(cause, E, N, maximal)
        if cause is not None:
            curr = tuple([cause[i] for i in idxs] + list(idxs))
            if curr in used:
                continue
            used.add(curr)
            H.append(cause)
            inxs.append(idxs)
    return zip(H, inxs)

Fi = algorithm(E, N, maximal=True)
print('Maximally CNF:')
print(' ∧ '.join(map(cause_str, Fi)))

Fi = algorithm(E, N, maximal=False)
print('Simplified CNF:')
print(' ∧ '.join(map(cause_str, Fi)))

Maximally CNF:
(p ∨ ¬q ∨ r) ∧ (¬p ∨ q ∨ r) ∧ (¬p ∨ ¬q ∨ r)
Simplified CNF:
(¬q ∨ r) ∧ (¬p ∨ r)


#### Answer:    
Algorithm 1/2; producing a maximally general set or simplified set of clauses H which covers E and avoids N. (3.3) in Logic_and_probability_7.pdf


1. φ with Maximally CNF: 

    (p ∨ ¬q ∨ r) ∧ (¬p ∨ q ∨ r) ∧ (¬p ∨ ¬q ∨ r)

2. φ with Simplified CNF: 

    (¬q ∨ r) ∧ (¬p ∨ r)


#### In the rest of the assignments L and M refer to the language L and L-structure M that we now define. 

Let L = {P,Q,R,S,T} where the relation symbols P,Q,R,S,T have arities 1,1,2,2,2, respectively.

Let M b e the L-structure with domain M = {1,2,3,...,20} and the following interpretations of the relation symbols:
* PM = {1,2,3,4,5,6,7,8,9,10},
* QM = {1,2,3,4,5,11,12,13,14,15},
* RM = {(i,j) : i ̸= j,1 ≤i ≤5 and 1 ≤j ≤5},
* SM = {(2,5),(3,1),(4,3),(1,2),(5,4),(2,3),(4,2),(1,3),(3,5),(4,5)},
* TM = {(2,5),(3,1),(4,3),(1,2),(5,4),(3,2),(2,1),(1,4),(2,4),(5,2)}.

#### Inductive logic programming and concept learning

* Would in it be fun to implement the algorithm that helps us to learn concepts? Yes!
* Can we use example in the in Logic_and_probability_7.pdf to guide and validate our implementation? Yes!
* Is it harder then just solving the assignments by hand? Yes!
* Is solving the assignments by hand a good idea? Probably Yes!
* Will we implement the algorithm even though it is harder and christmas is coming close? Yes!

Thank your patience. Lets get started!

PS. I will try to add some comments to the code to make it more readable. Don't skip them pls.

In [248]:
# lets define a base class to work with. it needs to have a name and arity
class Base:
    name = None
    arity = None
    def __repr__(self) -> str:
        return self.name + '(' + ','.join(map(str, self.args)) + ')'
    def __eq__(self, value: object) -> bool:
        return value.name == self.name and value.args == self.args
    def __hash__(self) -> int:
        return hash(self.name) + hash(self.args)

# how do we learn a concept?
def concept_learning(statement, domain, relations, structures, extra_variables=0, concept_size=2):
    # we need to find a concept that is true and false for all relation examples
    result = defaultdict(lambda: defaultdict(set))
    # We need to find a concept that has at least the same amount of variables as the arity as the statement
    variables = statement.arity
    # lets iterate over all possible concepts of size concept_size
    for structs in tqdm(list(combinations_with_replacement(structures, concept_size))):
        # lets se the combines arity of the concept we are currently exploring
        combined_arity = sum(map(lambda x: x.arity, structs))
        # lets give x0, x1, x2, ... xn to the statement. we can also check x0, x0, ... but that is not necessary
        init_var_stat = list(range(variables))
        # lets iterate possible initializations of the concept
        for init_var_concept in product(range(variables+extra_variables), repeat=combined_arity):
            # we need to make sure that the variables in the statement are also in the concept (otherwise we can learn crazy concepts)
            if not all(map(lambda x: x in init_var_concept, init_var_stat)):
                continue
            # lets build and abstract concept string
            concept_str = ' ∧ '.join(sorted(set([str(structs[i](*[f"x{j}" for j in init_var_concept[i:i+structs[i].arity]])) for i in range(len(structs))])))
            concept_str += ' <-> ' + str(statement(*[f"x{j}" for j in init_var_stat]))
            # lets iterate over all possible values for the variables
            for d in product(domain, repeat=variables):
                # lets initialize the statement
                stat = statement(*[d[j] for j in init_var_stat])
                # lets check if the statement is true or false for all relations
                condition_stat = stat in relations[stat.name]
                # there is allot of redundancy here so lets save the information little bit smarter (we can do better)
                valid = []
                examples = set()
                # if we have extra variables we need to iterate over them as well
                for e in product([dom for dom in domain if dom not in d], repeat=extra_variables):
                    cd = d + e
                    # lets initialize the concept
                    concept = [structs[i](*[cd[j] for j in init_var_concept[i:i+structs[i].arity]]) for i in range(len(structs))]
                    # lets check if the concept is true or false for all relations
                    condition_concept = list(map(lambda x: x in relations[x.name], concept))
                    # lets save the example
                    examples.add((stat, all(condition_concept) == condition_stat))
                    # lets check if the concept are equivalent
                    valid.append(all(condition_concept) == condition_stat)
                    
                result[concept_str]["truth"].add(all(valid))
                result[concept_str]["examples"].update(examples)
    # we are know have a dictionary with all possible concepts and their truth value and examples
    return result

# how do we read what concepts we have learned?
def parse_result(result, relations):
    concepts = []
    # lets iterate over all concepts
    for k, v in result.items():
        truth = v["truth"]
        examples = v["examples"]
        # if there latest one set of examples is true then we have and implication  
        if any(truth):
            valid = True
            cls, val = examples.pop()
            # but it its true for all relations then we have a equivalence (i think)
            for statement in relations[cls.name]:
                if (statement, True) not in examples:
                    valid = False
                    break
            if valid:
                concepts.append(k)
    # WOW! we have learned some concepts
    return concepts

#### Lets start by solving the assignments in the Logic_and_probability_7.pdf (3.4.1) 

In [241]:
class F(Base):
    arity = 1
    name = "F"
    def __init__(self, *args):
        self.args = args
        assert len(args) == F.arity
    
class P(Base):
    arity = 2
    name = "P"
    def __init__(self, *args):
        self.args = args
        assert len(args) == P.arity

class D(Base):
    arity = 2
    name = "D"
    def __init__(self, *args):
        self.args = args
        assert len(args) == D.arity

class G(Base):
    arity = 2
    name = "G"
    def __init__(self, *args):
        self.args = args
        assert len(args) == G.arity

A = ['a', 'b', 'e', 'g', 'h', 'm', 'n', 't']

relations = {'F': [F('m'), F('h'), F('e'), F('n')], # (i.e. m,n,e, and n are females),
             'P': [P('g', 'm'), P('h', 'm'), P('h', 't'), P('t', 'e'), P('n', 'e'), P('a', 't'), P('e', 'b')], # (i.e. g is a parent of m, h is a parent of m, and so on)
             'D': [D('g', 'm'), D('h', 'm'), D('t', 'e'), D('n', 'e')], # (i.e. g is a daughter of m, h is a daughter of m, and so on)
             'G': [G('h', 'e'), G('n', 'b')], # (i.e. h is a grandmother of e, n is a grandmother of b)
}


result = concept_learning(D, A, relations, [F, P, G], extra_variables=0, concept_size=2)
concepts = parse_result(result, relations)
print('Concepts: Daughters')

for c in concepts:
    print(c)

result = concept_learning(G, A, relations, [F, P, D], extra_variables=1, concept_size=3)
concepts = parse_result(result, relations)
print('Concepts: Grandmothers')

for c in concepts:
    print(c)

100%|██████████| 6/6 [00:00<00:00, 324.07it/s]


Concepts: Daughters
F(x1) ∧ P(x0,x1) <-> D(x0,x1)


100%|██████████| 10/10 [00:04<00:00,  2.09it/s]

Concepts: Grandmothers
F(x0) ∧ P(x2,x1) <-> G(x0,x1)
F(x0) ∧ F(x2) ∧ P(x2,x1) <-> G(x0,x1)
F(x0) ∧ P(x0,x2) ∧ P(x2,x1) <-> G(x0,x1)


Result:

We faund the following concept for Daughters:
1. F(x1) ∧ P(x0,x1) <-> D(x0,x1)

This makes sense witch is good!

We faund the following concept for Grandmothers:

1. F(x0) ∧ P(x2,x1) <-> G(x0,x1)
2. F(x0) ∧ F(x2) ∧ P(x2,x1) <-> G(x0,x1)
3. F(x0) ∧ P(x0,x2) ∧ P(x2,x1) <-> G(x0,x1)

To be honest i don't understand the first and second concept. The third concept makes me happy inside.
when adding extra variables to the concept it becomes more general. 

I can not argue that the first concept is wrong because if you are a female and you are a grandmother  then there exist a child and that child has a parent that you are a grandmother to. 

The same argument holds for the second concept its just that the parent of the child is a mother.

There exist counter examples for all concepts with the extra variable. However this holds for all concepts 

$$
A|= ∀x∀y∀z(((F(x) ∧ P(z,y))→G(x,y))\ and\ A|= ∃x∃y∃z((F(x) ∧ P(z,y)).
$$
$$
A|= ∀x∀y∀z(((F(x) ∧ P(z,y)) ↔G(x,y)).
$$


$$
A|= ∀x∀y∀z(((F(x) ∧ F(z) ∧ P(z,y))→G(x,y))\ and\ A|= ∃x∃y∃z((F(x) ∧ F(z) ∧ P(z,y)).
$$
$$
A|= ∀x∀y∀z(((F(x) ∧ F(z) ∧ P(z,y)) ↔G(x,y)).
$$


$$
A|= ∀x∀y∀z(((F(x) ∧ P(x,z) ∧ P(z,y))→G(x,y))\ and\ A|= ∃x∃y∃z((F(x) ∧ P(x,z) ∧ P(z,y)).
$$
$$
A|= ∀x∀y∀z(((F(x) ∧ P(x,z) ∧ P(z,y) ) ↔G(x,y)).
$$

Im not sure if this is the right way to argue but feel free to correct me!


#### Assignment 2

Learning the concept R(x,y): Find literals θ1(x,y),...,θn(x,y) which do not contain R and such that M |= ∀x∀y((θ1(x,y)∧...∧θn(x,y)) → R(x,y)) and M|= ∃x∃y(θ1(x,y)∧...∧θn(x,y)). Observe that a literal denoted θk(x,y) above need not necessarily contain both variables x and y and it may be of the form x = y or ¬(x = y).
There may be several possibilities, but to get maximal credit on this assignment you
need to find such θ1(x,y),...,θn(x,y) so that 

$$
M|= ∀x∀y((θ1(x,y) ∧... ∧θn(x,y)) ↔R(x,y))
$$

In [249]:
class P(Base):
    name = "P"
    arity = 1
    def __init__(self, *args):
        self.args = args
        assert len(args) == self.arity
    
class Q(Base):
    name = "Q"
    arity = 1
    def __init__(self, *args):
        self.args = args
        assert len(args) == self.arity

class R(Base):
    name = "R"
    arity = 2
    def __init__(self, *args):
        self.args = args
        assert len(args) == self.arity

class S(Base):
    name = "S"
    arity = 2
    def __init__(self, *args):
        self.args = args
        assert len(args) == self.arity

class T(Base):
    name = "T"
    arity = 2
    def __init__(self, *args):
        self.args = args
        assert len(args) == self.arity
        
M = list(range(1,16)) # we dont need to check for all 20 objects in the domain since we know that 1-15 are in the structure
PM = list(map(P, [1,2,3,4,5,6,7,8,9,10]))
QM = list(map(Q, [1,2,3,4,5,11,12,13,14,15]))
RM = [R(i,j) for i in range(1,6) for j in range(1,6) if i != j]
SM = list(map(S, [2,3,4,1,5,2,4,1,3,4], [5,1,3,2,4,3,2,3,5,5]))
TM = list(map(T, [2,3,4,1,5,3,2,1,2,5], [5,1,3,2,4,2,1,4,4,2]))
relations = {'P': PM, 'Q': QM, 'R': RM, 'S': SM, 'T': TM}

result = concept_learning(R, M, relations, [P, Q, S, T], extra_variables=2, concept_size=4)
concepts = parse_result(result, relations)
print('Concepts: R')
for c in concepts:
    print(c)


 20%|██        | 7/35 [07:53<38:41, 82.91s/it]

In [247]:
for p in product([1], [2,3], [1,2,3]):
    print(p)

(1, 2, 1)
(1, 2, 2)
(1, 2, 3)
(1, 3, 1)
(1, 3, 2)
(1, 3, 3)
